In [2]:
import json
import numpy as np
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.chat_models import ChatOpenAI

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from chatsky_llm_autoconfig.graph import BaseGraph, Graph
from chatsky_llm_autoconfig.metrics.embedder import get_embedding
from chatsky_llm_autoconfig.metrics.automatic_metrics import is_same_structure, llm_match
from chatsky_llm_autoconfig.schemas import CompareResponse
from chatsky_llm_autoconfig.algorithms.three_stages_graph_generation import ThreeStagesGraphGenerator
from chatsky_llm_autoconfig.dialogue import Dialogue

/home/peshkichev/miniconda3/envs/py11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(
/data/home/peshkichev/projects/chatsky-llm-autoconfig/dev_packages/chatsky_llm_autoconfig/chatsky_llm_autoconfig/algorithms/three_stages_graph_generation.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=env_settings.EMBEDDER_MODEL, model_kwargs={"device": env_settings.EMBEDDER_DEVICE})


In [5]:
from settings import EnvSettings
from utils import call_llm_api, read_json
from compare_prompt import (
    compare_graphs_prompt, graph_example_1, result_form
)

BAAI/bge-m3


In [6]:
env_settings = EnvSettings()


In [7]:
evaluator = HuggingFaceCrossEncoder(model_name=env_settings.RERANKER_MODEL, model_kwargs={"device": env_settings.EMBEDDER_DEVICE})
graph_generator = ThreeStagesGraphGenerator()

In [13]:
env_settings.EMBEDDER_MODEL

'BAAI/bge-m3'

In [22]:
env_settings.SIM_THRESHOLD

0.99

In [28]:
env_settings.EMBEDDER_TYPO

0.995

In [8]:
def read_json(path):
    with open(path, mode="r") as file:
        data = file.read()
    return json.loads(data)

In [22]:
%pwd

'/data/home/peshkichev/projects/chatsky-llm-autoconfig/experiments/exp20250207_llm_match'

In [9]:
dialogue_to_graph = read_json("../2025.01.22_sampler/data/generated_25.json")
results = read_json("../../dev_packages/chatsky_llm_autoconfig/chatsky_llm_autoconfig/autometrics/test_data/saved.json")["o1-mini"]["Feb_10"]

In [10]:
import logging
logging.getLogger("langchain_core.vectorstores.base").setLevel(logging.ERROR)

In [19]:
len(dialogue_to_graph)

91

In [36]:
case = dialogue_to_graph[88]
result_graph = graph_generator.invoke([Dialogue.from_list(c["messages"]) for c in case["dialogues"]])
llm_match(Graph(case['graph']),result_graph)

LISTS_N:  [(0, ["I'd like to check my baggage allowance for my international flight."]), (1, ["I'm flying with Delta in Economy class."]), (2, ["I'd like to add an additional bag.", 'Do you have information on carry-on baggage?', 'Actually, can I change my class to Business?', 'I would like to upgrade to Business class.']), (3, ['Actually, can I change my class to Business?', 'Yes, please add the extra bag.']), (4, ['Yes, that works.']), (5, ['Yes, please tell me about carry-on baggage.']), (6, ["No, that's all. Thank you!"]), (7, ['']), (8, ['Do you have information on carry-on baggage?', "I'd like to add an additional bag.", 'Actually, can I change my class to Business?', 'I would like to upgrade to Business class.']), (9, ["No, that's all. Thank you!"])]
LISTS:  [(0, 'Hello! How can I assist you today?'), (1, 'Sure! Could you please provide your airline and the class of your ticket?'), (2, 'Delta Economy class on international flights includes 1 checked luggage up to 23kg. Do you wa

True

In [21]:
case['graph']

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ['I need help recovering deleted voicemail messages.']},
  {'source': 2,
   'target': 3,
   'utterances': ['I accidentally deleted some voicemails from my phone.']},
  {'source': 3,
   'target': 4,
   'utterances': ["I'm using an iPhone.",
    'Actually, I use an Android phone.']},
  {'source': 4,
   'target': 5,
   'utterances': ["No, I haven't checked that yet."]},
  {'source': 5,
   'target': 6,
   'utterances': ['I found them! Can I restore them?']},
  {'source': 6, 'target': 7, 'utterances': ['Yes, please.']},
  {'source': 3,
   'target': 8,
   'utterances': ['Actually, I use an Android phone.']},
  {'source': 8,
   'target': 5,
   'utterances': ['Yes, I checked the trash and found them.']},
  {'source': 6,
   'target': 9,
   'utterances': ["Actually, I don't need help anymore."]},
  {'source': 5,
   'target': 9,
   'utterances': ["I changed my mind, I don't need to recover voicemails."]}],
 'nodes': [{'id': 1,
   'label': 

In [32]:
is_same_structure(Graph(case['graph']),Graph(result_graph))

True

In [16]:
llm_match(Graph(case['graph']),Graph(result_graph))

LEN1:  [1, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1] LEN2:  [1, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]
N_MAX:  [1, 2, 3, 4, 5, 7, 6, 8, 9, 0]
MAX:  [1, 2, 3, 4, 5, 7, 6, 8, 9, 0]
NODES:  0.9999999
ALL:  0.99618053


True

In [26]:
case['graph']

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ['I want to transfer data to a new phone.']},
  {'source': 2,
   'target': 3,
   'utterances': ["I'm transferring from an Android.",
    'I have an Android phone.',
    "It's an Android device."]},
  {'source': 2,
   'target': 6,
   'utterances': ["I'm transferring from an iPhone."]},
  {'source': 2,
   'target': 10,
   'utterances': ['I changed my mind, never mind.']},
  {'source': 3,
   'target': 4,
   'utterances': ["I'll use a cable.", "I'd like to use a cable."]},
  {'source': 3,
   'target': 5,
   'utterances': ["I'll transfer wirelessly.",
    "I'd like to transfer wirelessly."]},
  {'source': 4, 'target': 7, 'utterances': ["I've connected the phones."]},
  {'source': 4,
   'target': 3,
   'utterances': ['Actually, can I change to wireless instead.']},
  {'source': 5, 'target': 7, 'utterances': ['Yes, proceed.']},
  {'source': 5,
   'target': 3,
   'utterances': ["Actually, I'd like to use a cable instead."]},
  {'source'

In [22]:
llm_match(Graph(result_graph), Graph(case['graph']))

LEN1:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] LEN2:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
MAX:  [9, 0, 1, 2, 4, 5, 7, 3, 6, 8, 10]
N_MAX:  [9, 0, 1, 2, 4, 5, 7, 3, 6, 8, 3]
LLLLENS


False

In [21]:
llm_match(Graph(case['graph']),Graph(result_graph))

LEN1:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] LEN2:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
MAX:  [1, 2, 3, 7, 4, 5, 8, 6, 9, 0, 10]
N_MAX:  [1, 2, 3, 7, 4, 5, 8, 6, 9, 0, 10]
NODES:  0.970732
ALL:  0.9805123


True

In [15]:
case['graph']

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ['I need braille documents.',
    'I have a question about braille documents.']},
  {'source': 2, 'target': 3, 'utterances': ['I need the annual report.']},
  {'source': 2,
   'target': 7,
   'utterances': ['Actually, I want a different document.']},
  {'source': 7, 'target': 3, 'utterances': ["I'd like the user guide."]},
  {'source': 3, 'target': 4, 'utterances': ['Email, please.']},
  {'source': 3,
   'target': 8,
   'utterances': ["Actually, I'd like to download it instead."]},
  {'source': 8, 'target': 6, 'utterances': ['Download, please.']},
  {'source': 4, 'target': 5, 'utterances': ['example@example.com.']},
  {'source': 5, 'target': 6, 'utterances': ["No, that's all."]},
  {'source': 2,
   'target': 6,
   'utterances': ["Nevermind, I don't need it anymore."]},
  {'source': 3,
   'target': 6,
   'utterances': ["Actually, I don't need the document."]},
  {'source': 7, 'target': 6, 'utterances': ['I changed my mind.']},
  

In [ ]:
dic = [d for d in results if list(d.keys())[0]=="Reminding about renewal deadlines.Sending service expiration alerts."][0]["Reminding about renewal deadlines.Sending service expiration alerts."][0]
result_graph = {"nodes": dic['nodes'], "edges": dic['edges']}
case = [d for d in dialogue_to_graph if d['topic']=="Reminding about renewal deadlines.Sending service expiration alerts."][0]
llm_match(Graph(case['graph']),Graph(result_graph))

FIRST:  8 9


False

In [12]:
result_graph

{'nodes': [{'id': 1,
   'label': '',
   'is_start': False,
   'utterances': ["Excellent! Your renewal reminders are set to 30 days prior to the deadline and will be sent via SMS. Is there anything more you'd like to configure?",
    'Great! You will receive renewal reminders 30 days before the deadline via SMS. Would you like to set up anything else?',
    "Excellent! Your renewal reminders are set to 14 days prior to the deadline and will be sent via email. Is there anything more you'd like to configure?"]},
  {'id': 2,
   'label': '',
   'is_start': True,
   'utterances': ["Hello! I'm here to help you manage your service renewals. How can I assist you today?"]},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ['Sure! When would you like to receive the reminders? Please specify the number of days before the renewal deadline.']},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Got it. How would you like to receive these reminders? Via email, SMS,

In [ ]:
case = [d for d in dialogue_to_graph if d['topic']=="Exporting chat history from messaging apps."][0]
result_graph1 = graph_generator.invoke([Dialogue.from_list(c["messages"]) for c in case["dialogues"]])
llm_match(Graph(case['graph']),result_graph1)


LISTS_N:  [(0, ['I would like to export my chat history.']), (1, ["I'm using WhatsApp."]), (2, ['Without media.', 'With media.']), (3, ['Email.', 'Save to cloud storage.']), (4, ['Yes, I need something else.', 'No, thank you.']), (5, ['I want to export another chat.']), (6, ['Save to cloud storage.', 'Actually, can I change my media option?', 'Actually, can I change my delivery option?', 'Email.']), (7, ['No, thank you.', 'Yes, I need something else.']), (8, ['']), (9, ["Sure, I'd like to export without media."]), (10, ["Sure, I'd like to export without media."]), (11, ['Email.', 'Save to cloud storage.'])]
LISTS:  [(0, 'Hello! How can I assist you today?'), (1, 'Sure, which messaging app are you using?'), (2, 'Would you like to export with or without media?'), (3, 'Understood. How would you like to receive your chat history? Via email or save to your cloud storage?'), (4, 'Your chat history is being sent to your email. Is there anything else I can help you with?'), (5, 'Sure, what els

True

In [22]:
result_graph1.graph_dict

{'nodes': [{'id': 1,
   'label': '',
   'is_start': False,
   'utterances': ['Certainly! Do you want to include media in your export?',
    'Of course! Would you like to export with or without media?']},
  {'id': 2,
   'label': '',
   'is_start': True,
   'utterances': ['Hello! How can I assist you today?']},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ['Sure, which messaging app are you using?']},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Would you like to export with or without media?']},
  {'id': 5,
   'label': '',
   'is_start': False,
   'utterances': ['Understood. How would you like to receive your chat history? Via email or save to your cloud storage?']},
  {'id': 6,
   'label': '',
   'is_start': False,
   'utterances': ['Your chat history is being sent to your email. Is there anything else I can help you with?']},
  {'id': 7,
   'label': '',
   'is_start': False,
   'utterances': ['Sure, what else would you like assistance with

In [19]:
is_same_structure(Graph(case['graph']),result_graph)

True

In [20]:
result_graph.graph_dict

{'nodes': [{'id': 1,
   'label': '',
   'is_start': False,
   'utterances': ['Alright, let me know if you need anything else. Have a great day!',
    'Okay, feel free to reach out if you need further assistance!']},
  {'id': 2,
   'label': '',
   'is_start': True,
   'utterances': ['Greetings! What would you like to do today?',
    'Hello! How can I assist you today?',
    'Hi there! What can I help you with today?']},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ['Sure, I can help with that. Are you transferring from an Android or an iPhone?']},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Great. Do you want to use a cable or transfer wirelessly?']},
  {'id': 5,
   'label': '',
   'is_start': False,
   'utterances': ["Please connect both phones using a compatible cable. Let me know once they're connected."]},
  {'id': 6,
   'label': '',
   'is_start': False,
   'utterances': ['Ensure both phones are connected to the same Wi-Fi network and 

In [34]:
result_graph = graph_generator.invoke([Dialogue.from_list(c["messages"]) for c in case["dialogues"]])

LISTS_N:  [(0, ["My cart won't let me proceed to checkout.", "I'm getting an error when I try to add items to the cart.", 'The checkout page is not loading.', 'My cart keeps crashing when I try to add items.']), (1, ['Actually, can I change my browser?', "No, I haven't tried that.", "Yes, but it didn't work."]), (2, ['I want to try Safari instead.']), (3, ['It works now.', 'Still having issues.']), (4, ["No, that's all."]), (5, ['']), (6, ['It works now.', 'Still having issues.']), (7, ["My cart won't let me proceed to checkout.", "I'm getting an error when I try to add items to the cart.", 'The checkout page is not loading.', 'My cart keeps crashing when I try to add items.']), (8, ["I'm using Chrome on Windows."]), (9, ["Yes, but it didn't work.", 'Actually, can I change my device?', "No, I haven't tried that."]), (10, ["It still doesn't work.", 'It worked! Thank you.']), (11, ['']), (12, ["No, I haven't cleared cache and cookies.", 'Can I change my device?', "Yes, I cleared cache an

In [25]:
result_graph.graph_dict

{'nodes': [{'id': 1,
   'label': '',
   'is_start': False,
   'utterances': ['Of course, we can facilitate an exchange for a different size. Please specify your preferred size.',
    'Certainly, we can exchange your product for a different size. Please let us know which size you would prefer.',
    'Sure, we can exchange your product for another size. Which size would you like instead?']},
  {'id': 2,
   'label': '',
   'is_start': True,
   'utterances': ["I'm sorry to hear that the size of the product you received isn't what you expected. How can I assist you today?"]},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ["I'm sorry for the inconvenience. Could you please provide your order number?"]},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Thank you. I see that you ordered a Medium size. Would you like a replacement, a refund, or to exchange it for a different size?']},
  {'id': 5,
   'label': '',
   'is_start': False,
   'utterances': ['U

In [35]:
llm_match(Graph(case['graph']),result_graph)

LEN1:  [1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] LEN2:  [1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
N_MAX:  [2, 3, 7, 11, 9, 4, 10, 12, 0, 8, 1, 5, 6]
MAX:  [2, 3, 7, 11, 9, 4, 10, 12, 0, 8, 1, 5, 6]
MAPS:  {1: 3, 2: 4, 3: 8, 4: 12, 5: 10, 6: 5, 7: 11, 8: 13, 9: 1, 10: 9, 11: 2, 12: 6, 13: 7}
NN:  1 3
NN:  2 4
NN:  3 8
NN:  4 12
NN:  5 10
NN:  6 5
NN:  7 11
NN:  8 13
NN:  9 1
NN:  10 9
NN:  11 2
NN:  12 6
NN:  13 7
NODES:  0.99999964
ALL:  0.98246884
MIN:  0.98246884


True

In [29]:
case = dialogue_to_graph[20]
result_graph = graph_generator.invoke([Dialogue.from_list(c["messages"]) for c in case["dialogues"]])
llm_match(Graph(dialogue_to_graph[20]['graph']),result_graph)

0.99
LISTS_N:  [(0, ['Yes, I have changed the OS.', "No, I haven't changed the OS."]), (1, ["I've enabled it, but it's still not working."]), (2, ["I've done that, but the problem persists.", 'Actually, I want to restart my device.']), (3, ["No, I'd like to keep trying biometric login.", "Yes, I'll use my password."]), (4, ["I've restarted it, but it still doesn't work.", 'Actually, I realized my fingerprint scanner is dirty. Let me clean it and try again.']), (5, ['']), (6, ["Yes, I'll use my password.", "No, I'd like to keep trying biometric login."]), (7, ['I want to try logging in with my password instead.', "I've verified it, but it still doesn't work.", 'It still fails after verification.']), (8, ['']), (9, ["I restarted it, but it still doesn't work."])]
LISTS:  [(0, "I'm sorry you're having trouble with biometric login. Let's troubleshoot together. Have you recently updated your device's operating system?"), (1, "Thank you for letting me know. Sometimes OS updates can affect se

True

In [113]:
llm_match(Graph(dialogue_to_graph[15]['graph']),result_graph)

LEN1:  [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1] LEN2:  [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
N_MAX:  [1, 2, 3, 6, 4, 7, 8, 0, 5]
MAX:  [1, 2, 3, 6, 4, 7, 8, 0, 5]
MAPS:  {1: 2, 2: 3, 3: 4, 4: 7, 5: 5, 6: 8, 7: 9, 8: 1, 9: 6}
NN:  1 2
NN:  2 3
NN:  3 4
NN:  4 7
NN:  5 5
NN:  6 8
NN:  7 9
NN:  8 1
NN:  9 6
NODES:  0.9898924
ALL:  0.9870714
MIN:  0.9870714


True

In [118]:
is_same_structure(Graph(dialogue_to_graph[17]['graph']),result_graph)

False

In [80]:
def remove_duplicated_edges(graph: dict):
    edges = graph['edges']
    couples = [(e['source'],e['target']) for e in edges]
    duplicates = [i for i in set(couples) if couples.count(i) > 1]
    print(duplicates)
    new_edges = []
    for d in duplicates:
        found = [c for c in edges if c['source'] == d[0] and c['target'] == d[1]]
        print("FOUND: ", found)
        new_edge = found[0]
        new_edge['utterances'] = []
        for e in found:
            new_edge['utterances'].extend(e['utterances'])
        new_edges.append(new_edge)
    new_graph = {"edges": [e for e in edges if (e['source'],e['target']) not in duplicates]+new_edges, "nodes": graph['nodes'] }
    return new_graph

In [24]:
result_graph.graph_dict

{'nodes': [{'id': 1,
   'label': '',
   'is_start': False,
   'utterances': ['Great! Your replacement card will be mailed to your address within 5 business days. Is there anything else I can help you with?',
    "Excellent! We'll send your new membership card to your registered address and it should arrive in about 5 days. Do you need anything else?"]},
  {'id': 2,
   'label': '',
   'is_start': True,
   'utterances': ['Hello! How can I assist you today?']},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ['Sure, I can help with that. Could you please provide your membership number?']},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Thank you. How would you like to receive your replacement card? It can be mailed to your address on file or you can pick it up at any of our stores.']},
  {'id': 5,
   'label': '',
   'is_start': False,
   'utterances': ['Of course! You can pick up your replacement card at the nearest store within 3 business days. Is

In [17]:
dialogue_to_graph[17]['topic']

'Requesting a replacement membership card'

In [16]:
dialogue_to_graph[17]['graph']

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ['I need a replacement membership card.']},
  {'source': 2,
   'target': 3,
   'utterances': ['My membership number is 123456.']},
  {'source': 3,
   'target': 4,
   'utterances': ['I would like it mailed to my address.']},
  {'source': 3,
   'target': 5,
   'utterances': ['I prefer to pick it up at a store.']},
  {'source': 4,
   'target': 7,
   'utterances': ['Actually, can I change to picking it up instead?',
    'Can I switch to store pickup instead?',
    'I would like to change my delivery method to pickup.']},
  {'source': 4, 'target': 6, 'utterances': ["No, that's all."]},
  {'source': 5, 'target': 6, 'utterances': ['No, thank you.']},
  {'source': 5,
   'target': 8,
   'utterances': ['Actually, I would like it mailed instead.']},
  {'source': 8, 'target': 6, 'utterances': ["No, that's all."]},
  {'source': 2, 'target': 9, 'utterances': ['Actually, I changed my mind.']},
  {'source': 7,
   'target': 8,
   'utterances': [

In [ ]:
dialogue_to_graph[7]['']

In [52]:
list(results[7].values())[0][0]

{'nodes': [{'id': 1,
   'label': '',
   'is_start': True,
   'utterances': ["I'm sorry you're experiencing issues with your checkout cart. Can you please describe the problem?",
    "Sorry to hear you're having trouble with your checkout cart. Could you please explain what's happening?"]},
  {'id': 2,
   'label': '',
   'is_start': False,
   'utterances': ['Great to hear! Is there anything else I can help you with?',
    "You're welcome! Is there anything else I can help you with?",
    'Glad to hear that! Do you need any further assistance?']},
  {'id': 3,
   'label': '',
   'is_start': False,
   'utterances': ['Please try refreshing the page first and let me know if the issue persists.',
    'Please try using Firefox and let me know if the issue persists.']},
  {'id': 4,
   'label': '',
   'is_start': False,
   'utterances': ['Have you tried using a different browser or device?',
    "Okay, have you cleared your browser's cache and cookies?"]},
  {'id': 5,
   'label': '',
   'is_star

In [91]:
dialogue_to_graph[6]['graph']

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ['I need to report an incorrect product dimension listing.',
    "There's an issue with a product's dimensions."]},
  {'source': 1,
   'target': 6,
   'utterances': ["Never mind, I don't need help right now."]},
  {'source': 2, 'target': 3, 'utterances': ['The product ID is 12345.']},
  {'source': 2,
   'target': 4,
   'utterances': ["I'd prefer to give you the product name instead."]},
  {'source': 3,
   'target': 5,
   'utterances': ['The length and width are wrong.']},
  {'source': 3,
   'target': 7,
   'utterances': ["Actually, I provided the product number incorrectly. It's 67890."]},
  {'source': 4, 'target': 5, 'utterances': ['The height is incorrect.']},
  {'source': 5, 'target': 6, 'utterances': ["No, that's all."]},
  {'source': 5,
   'target': 2,
   'utterances': ['Actually, I need to change the product ID.']},
  {'source': 7, 'target': 5, 'utterances': ['The height is wrong.']}],
 'nodes': [{'id': 1,
   'label': 'Ass

In [81]:
cur_graph = remove_duplicated_edges(dialogue_to_graph[7]['graph'])

[(6, 11)]
FOUND:  [{'source': 6, 'target': 11, 'utterances': ['I want to try Safari instead.']}, {'source': 6, 'target': 11, 'utterances': ['I want to try Safari instead.']}]


In [82]:
cur_graph

{'edges': [{'source': 1,
   'target': 2,
   'utterances': ["My cart won't let me proceed to checkout."]},
  {'source': 1,
   'target': 3,
   'utterances': ["I'm getting an error when I try to add items to the cart.",
    'The checkout page is not loading.',
    'My cart keeps crashing when I try to add items.']},
  {'source': 2, 'target': 4, 'utterances': ["Yes, but it didn't work."]},
  {'source': 2, 'target': 5, 'utterances': ["No, I haven't tried that."]},
  {'source': 2,
   'target': 6,
   'utterances': ['Actually, can I change my browser?']},
  {'source': 4,
   'target': 7,
   'utterances': ["Yes, I cleared cache and cookies, but it still doesn't work."]},
  {'source': 4,
   'target': 5,
   'utterances': ["No, I haven't cleared cache and cookies."]},
  {'source': 4, 'target': 8, 'utterances': ['Can I change my device?']},
  {'source': 5, 'target': 7, 'utterances': ["It still doesn't work."]},
  {'source': 5, 'target': 9, 'utterances': ['It worked! Thank you.']},
  {'source': 3, 't

In [83]:
llm_match(Graph(cur_graph),result_graph)

LEN1:  [1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] LEN2:  [1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
SCORING...


finished
MAX:  [2, 3, 7, 11, 9, 4, 10, 12, 0, 8, 1, 5, 6]
MAPS:  {1: 3, 2: 4, 3: 8, 4: 12, 5: 10, 6: 5, 7: 11, 8: 13, 9: 1, 10: 9, 11: 2, 12: 6, 13: 7}
NN:  1 3
NN:  2 4
NN:  3 8
NN:  4 12
NN:  5 10
NN:  6 5
NN:  7 11
NN:  8 13
NN:  9 1
NN:  10 9
NN:  11 2
NN:  12 6
NN:  13 7
NODES:  0.9999747
ALL:  0.99996185


True

In [19]:
def match(index):
    return llm_match(Graph(dialogue_to_graph[index]['graph']),Graph(list(results[index].values())[0][0]))

In [26]:
match(6)

FIRST


False